# Filter Orders

In [37]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import re
from IPython.display import display
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

## 1. Baca File CSV

In [38]:
# Membaca file jan_2025.csv dengan hanya mengambil kolom yang diperlukan
# untuk menghemat memori karena file sangat besar
selected_columns = [
    "idStore.orderNumber", 
    "general.orderCompleteTimestamp", 
    "customer.customerIdentifier", 
    "customer.customerName", 
    "general.status", 
    "general.orderCreateTimestamp", 
    "general.orderActiveTimestamp", 
    "general.orderSubmitTimestamp", 
    "general.orderType", 
    "general.productType", 
    "general.produced.area", 
    "collection.requestedVehicles.0", 
    "collection.collectionVehicles.0",
    "general.produced.geoJson.properties.centroidX",
    "general.produced.geoJson.properties.centroidY"
]
# Read CSV with selected columns
df_raw = pd.read_csv('raw.csv', usecols=selected_columns)
df_result = pd.read_csv('result.csv', sep=',', dtype={'Order Number': str}, na_filter=True).dropna(how='all')


In [39]:
def to_camel_case(s):
    s = re.sub(r'[^0-9a-zA-Z]+', ' ', s)         # ganti non-alphanumeric jadi spasi
    parts = s.title().replace(" ", "")           # kapital tiap kata lalu hilangin spasi
    return parts[0].lower() + parts[1:] if parts else s

In [40]:
# Membaca file jan_2025_result.csv (file dengan Order Notes)
# Perhatikan bahwa file ini menggunakan separator ';'
# Menampilkan beberapa baris pertama untuk memeriksa struktur data
print(f"Jumlah baris data diproses: {len(df_result)}")
# Filter data jan_2025_result untuk mendapatkan baris dengan Order Notes 'N'
df_result_f = df_result[(df_result['Order Notes'] == 'N')].copy()
df_result_f.columns = [to_camel_case(col) for col in df_result_f.columns]

# Menampilkan hasil filter
print(f"Jumlah baris dengan Order Notes 'N': {len(df_result_f)}")
# Format numbers to avoid scientific notation
display(df_result_f.tail())

Jumlah baris data diproses: 790
Jumlah baris dengan Order Notes 'N': 425


,archiveTasking,publicPrivate,multistrip,responsiveness,orderNumber,completeDate,orderDescription,catId,sqkm,charge,chargeS,orderNotes
781,ARCHIVE,Public,N,Standard,017347587010,11/1/2024,00:00:00,1040010089D0D200,24.99,$24.99,00:00:00.048,N
782,ARCHIVE,Public,N,Standard,017347588010,11/1/2024,00:00:00,1040010069A58200,25.03,$25.03,00:00:00.048,N
783,ARCHIVE,Public,N,Fresh,017347585010,11/1/2024,00:00:00,104001009E397400,25.01,$25.01,00:00:00.048,N
784,ARCHIVE,Public,N,Fresh,017347586010,11/1/2024,00:00:00,10300101067C5900,25.00,$25.00,00:00:00.054,N
789,ARCHIVE,Public,N,Fresh,050246772010,10/24/2024,View-Ready (Standard) OR2A,B11000110073A000,50.08,$50.08,00:00:00.135,N


In [61]:
df_mmerged_clean

,orderNumber,customerIdentifier,customerName,status,orderCreateTimestamp,orderActiveTimestamp,orderSubmitTimestamp,orderCompleteTimestamp,orderType,productType,...,multistrip,responsiveness,completeDate,orderDescription,catId,sqkm,charge,chargeS,orderNotes,country
8,50354788010,100436,DAF75,Complete,2025-07-28T07:54:49.0000 UTC,2025-07-28T07:55:27.6310 UTC,2025-07-28T07:54:49.0000 UTC,2025-07-28T08:00:06.3090 UTC,AO,OR2A,...,N,Standard,7/28/2025,View-Ready (Standard) OR2A,103001010B13CC00,12.56,12.56,0.027,N,Indonesia
9,50354785010,100436,DAF75,Complete,2025-07-28T07:49:55.0000 UTC,2025-07-28T07:50:20.6890 UTC,2025-07-28T07:49:55.0000 UTC,2025-07-28T07:56:09.1990 UTC,AO,OR2A,...,N,Standard,7/28/2025,View-Ready (Standard) OR2A,B120001100DBB200,12.56,12.56,0.034,N,Indonesia
10,50352143010,100436,DAF75,Complete,2025-07-18T09:24:33.0000 UTC,2025-07-18T09:29:45.5550 UTC,2025-07-18T09:24:33.0000 UTC,2025-07-18T09:32:26.2090 UTC,AO,OR2A,...,N,Fresh,7/18/2025,View-Ready (Standard) OR2A,1030010114745A00,3.86,3.86,0.008,N,Indonesia
11,50352134010,100436,DAF75,Complete,2025-07-18T08:04:41.0000 UTC,2025-07-18T08:05:16.5100 UTC,2025-07-18T08:04:41.0000 UTC,2025-07-18T08:09:38.0330 UTC,AO,OR2A,...,N,Standard,7/18/2025,View-Ready (Standard) OR2A,##############################################...,1.19,1.19,0.002,N,Indonesia
12,50351772010,100436,DAF75,Complete,2025-07-17T06:07:01.0000 UTC,2025-07-17T06:07:31.9980 UTC,2025-07-17T06:07:01.0000 UTC,2025-07-17T06:17:37.3730 UTC,AO,2A,...,N,Standard,7/17/2025,View-Ready (Standard) 2A,104001009C0D5800,79.99,79.99,0.155,N,Indonesia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,17347587010,100436,DAF75,Delivered,2024-10-31T08:59:05.0000 UTC,2024-10-31T08:59:46.0000 UTC,2024-10-31T08:59:05.0000 UTC,2024-11-01T05:02:54.0000 UTC,AO,L3,...,N,Standard,11/1/2024,00:00:00,1040010089D0D200,24.99,24.99,0.048,N,Singapore
579,17347588010,100436,DAF75,Delivered,2024-10-31T08:59:05.0000 UTC,2024-10-31T08:59:47.0000 UTC,2024-10-31T08:59:05.0000 UTC,2024-11-01T05:02:54.0000 UTC,AO,L3,...,N,Standard,11/1/2024,00:00:00,1040010069A58200,25.03,25.03,0.048,N,Indonesia
580,17347585010,100436,DAF75,Delivered,2024-10-31T08:58:56.0000 UTC,2024-10-31T08:59:51.0000 UTC,2024-10-31T08:58:56.0000 UTC,2024-11-01T05:03:13.0000 UTC,AO,L3,...,N,Fresh,11/1/2024,00:00:00,104001009E397400,25.01,25.01,0.048,N,Indonesia
581,17347586010,100436,DAF75,Delivered,2024-10-31T08:58:55.0000 UTC,2024-10-31T08:59:47.0000 UTC,2024-10-31T08:58:55.0000 UTC,2024-11-01T05:03:11.0000 UTC,AO,L3,...,N,Fresh,11/1/2024,00:00:00,10300101067C5900,25.00,25.00,0.054,N,Indonesia


In [41]:


# Define new column names mapping
new_column_names = {
    "idStore.orderNumber": "orderNumber",
    "general.orderCompleteTimestamp": "orderCompleteTimestamp",
    "customer.customerIdentifier": "customerIdentifier", 
    "customer.customerName": "customerName",
    "general.status": "status",
    "general.orderCreateTimestamp": "orderCreateTimestamp",
    "general.orderActiveTimestamp": "orderActiveTimestamp",
    "general.orderSubmitTimestamp": "orderSubmitTimestamp",
    "general.orderType": "orderType",
    "general.productType": "productType",
    "general.produced.area": "area",
    "collection.requestedVehicles.0": "requestedVehicles",
    "collection.collectionVehicles.0": "collectionVehicles",
    "general.produced.geoJson.properties.centroidX": "longitude",
    "general.produced.geoJson.properties.centroidY": "latitude"
}

# Rename columns using the mapping
df_raw.rename(columns=new_column_names, inplace=True)

# Menampilkan beberapa baris pertama untuk memeriksa struktur data
print(f"Data mentah: {len(df_raw)}")
display(df_raw.head())

Data mentah: 1000


,orderNumber,customerIdentifier,customerName,status,orderCreateTimestamp,orderActiveTimestamp,orderSubmitTimestamp,orderCompleteTimestamp,orderType,productType,area,requestedVehicles,longitude,latitude,collectionVehicles
0,17458887010,100436,DAF75,Active,2025-07-30T06:51:20.0000 UTC,2025-07-30T06:51:45.0000 UTC,2025-07-30T06:51:20.0000 UTC,No Data,SO,SPEC,0.000,WV03,No Data,No Data,No Data
1,17458886010,100436,DAF75,Complete,2025-07-30T06:35:47.0000 UTC,2025-07-30T06:36:14.0000 UTC,2025-07-30T06:35:47.0000 UTC,2025-08-02T04:52:49.0000 UTC,SO,SPEC,142.922,WV03,140.8754035,-2.8755395,GE01
2,17458885010,100436,DAF75,Active,2025-07-30T06:34:23.0000 UTC,2025-07-30T06:34:52.0000 UTC,2025-07-30T06:34:23.0000 UTC,No Data,SO,SPEC,0.000,WV03,No Data,No Data,No Data
3,17458884010,100436,DAF75,Complete,2025-07-30T06:29:47.0000 UTC,2025-07-30T06:30:05.0000 UTC,2025-07-30T06:29:47.0000 UTC,2025-08-08T04:53:02.0000 UTC,SO,SPEC,52.533,WV03,140.461232,-6.905749999999999,GE01
4,17458883010,100436,DAF75,Active,2025-07-30T06:28:38.0000 UTC,2025-07-30T06:28:58.0000 UTC,2025-07-30T06:28:38.0000 UTC,No Data,SO,SPEC,0.000,WV03,No Data,No Data,No Data


In [42]:
# Membaca file jan_2025.csv dengan hanya mengambil kolom yang diperlukan
# untuk menghemat memori karena file sangat besar
selected_columns = [
    "idStore.orderNumber", 
    "general.orderCompleteTimestamp", 
    "customer.customerIdentifier", 
    "customer.customerName", 
    "general.status", 
    "general.orderCreateTimestamp", 
    "general.orderActiveTimestamp", 
    "general.orderSubmitTimestamp", 
    "general.orderType", 
    "general.productType", 
    "general.produced.area", 
    "collection.requestedVehicles.0", 
    "collection.collectionVehicles.0",
    "general.produced.geoJson.properties.centroidX",
    "general.produced.geoJson.properties.centroidY"
]

# Define new column names mapping
new_column_names = {
    "idStore.orderNumber": "orderNumber",
    "general.orderCompleteTimestamp": "orderCompleteTimestamp",
    "customer.customerIdentifier": "customerIdentifier", 
    "customer.customerName": "customerName",
    "general.status": "status",
    "general.orderCreateTimestamp": "orderCreateTimestamp",
    "general.orderActiveTimestamp": "orderActiveTimestamp",
    "general.orderSubmitTimestamp": "orderSubmitTimestamp",
    "general.orderType": "orderType",
    "general.productType": "productType",
    "general.produced.area": "area",
    "collection.requestedVehicles.0": "requestedVehicles",
    "collection.collectionVehicles.0": "collectionVehicles",
    "general.produced.geoJson.properties.centroidX": "longitude",
    "general.produced.geoJson.properties.centroidY": "latitude"
}

# Rename columns using the mapping
df_raw.rename(columns=new_column_names, inplace=True)

# Menampilkan beberapa baris pertama untuk memeriksa struktur data
print(f"Data mentah: {len(df_raw)}")
display(df_raw.head())

Data mentah: 1000


,orderNumber,customerIdentifier,customerName,status,orderCreateTimestamp,orderActiveTimestamp,orderSubmitTimestamp,orderCompleteTimestamp,orderType,productType,area,requestedVehicles,longitude,latitude,collectionVehicles
0,17458887010,100436,DAF75,Active,2025-07-30T06:51:20.0000 UTC,2025-07-30T06:51:45.0000 UTC,2025-07-30T06:51:20.0000 UTC,No Data,SO,SPEC,0.000,WV03,No Data,No Data,No Data
1,17458886010,100436,DAF75,Complete,2025-07-30T06:35:47.0000 UTC,2025-07-30T06:36:14.0000 UTC,2025-07-30T06:35:47.0000 UTC,2025-08-02T04:52:49.0000 UTC,SO,SPEC,142.922,WV03,140.8754035,-2.8755395,GE01
2,17458885010,100436,DAF75,Active,2025-07-30T06:34:23.0000 UTC,2025-07-30T06:34:52.0000 UTC,2025-07-30T06:34:23.0000 UTC,No Data,SO,SPEC,0.000,WV03,No Data,No Data,No Data
3,17458884010,100436,DAF75,Complete,2025-07-30T06:29:47.0000 UTC,2025-07-30T06:30:05.0000 UTC,2025-07-30T06:29:47.0000 UTC,2025-08-08T04:53:02.0000 UTC,SO,SPEC,52.533,WV03,140.461232,-6.905749999999999,GE01
4,17458883010,100436,DAF75,Active,2025-07-30T06:28:38.0000 UTC,2025-07-30T06:28:58.0000 UTC,2025-07-30T06:28:38.0000 UTC,No Data,SO,SPEC,0.000,WV03,No Data,No Data,No Data


## 2. Optimasi Pencarian

Metode di atas mungkin lambat untuk dataset besar. Berikut adalah pendekatan yang lebih efisien.

In [43]:
# Dapatkan daftar Order Number dengan Order Notes 'N'
def clean_number(x):
    # Convert to string first to handle any numeric inputs
    x = str(x)
    # Remove leading/trailing zeros by converting to int and back to string
    try:
        return str(int(x))
    except ValueError:
        return x

# Fungsi untuk memeriksa apakah baris harus dipertahankan
def should_keep_row(row):
    # Jika status bukan 'Complete', langsung pertahankan
    if row['status'] != 'Complete':
        return True
    # Jika status 'Complete', periksa apakah idStore.orderNumber mengandung Order Number dengan Notes 'N'
    else:
        idstore_number = row['orderNumber']
        # Check for exact match instead of partial match
        if idstore_number in order_numbers_with_n:
            return True
        
        return False

df_result_f.loc[:, 'orderNumber'] = df_result_f['orderNumber'].astype(str).apply(clean_number)
order_numbers_with_n = df_result_f['orderNumber'].astype(str).apply(clean_number).tolist()
df_raw['orderNumber'] = df_raw['orderNumber'].astype(str).apply(clean_number)

# Print debug information
print(f"Total orders with Note 'N': {len(order_numbers_with_n)}")
print(f"Total orders status except 'Complete': {len(df_raw[df_raw['status'] != 'Complete'])}")
print(f"Total orders before filtering: {len(df_raw)}")
# print(f"Sample order numbers with N: {order_numbers_with_n[:5]}")
# print("\nSample idStore.orderNumber values:")
# print(df_jan['idStore.orderNumber_str'].head())

# Terapkan fungsi ke DataFrame
df_raw['keep_row'] = df_raw.apply(should_keep_row, axis=1)

# Filter baris yang harus dipertahankan
df_matching = df_raw[df_raw['keep_row']].copy()
# Hapus kolom tambahan
df_matching.drop(['keep_row'], axis=1, inplace=True)

# Menampilkan hasil
print(f"Total orders after filtering:: {len(df_matching)}")
display(df_matching.head())

Total orders with Note 'N': 425
Total orders status except 'Complete': 176
Total orders before filtering: 1000
Total orders after filtering:: 583


,orderNumber,customerIdentifier,customerName,status,orderCreateTimestamp,orderActiveTimestamp,orderSubmitTimestamp,orderCompleteTimestamp,orderType,productType,area,requestedVehicles,longitude,latitude,collectionVehicles
0,17458887010,100436,DAF75,Active,2025-07-30T06:51:20.0000 UTC,2025-07-30T06:51:45.0000 UTC,2025-07-30T06:51:20.0000 UTC,No Data,SO,SPEC,0.0,WV03,No Data,No Data,No Data
2,17458885010,100436,DAF75,Active,2025-07-30T06:34:23.0000 UTC,2025-07-30T06:34:52.0000 UTC,2025-07-30T06:34:23.0000 UTC,No Data,SO,SPEC,0.0,WV03,No Data,No Data,No Data
4,17458883010,100436,DAF75,Active,2025-07-30T06:28:38.0000 UTC,2025-07-30T06:28:58.0000 UTC,2025-07-30T06:28:38.0000 UTC,No Data,SO,SPEC,0.0,WV03,No Data,No Data,No Data
5,17458882010,100436,DAF75,Active,2025-07-30T06:25:49.0000 UTC,2025-07-30T06:26:13.0000 UTC,2025-07-30T06:25:49.0000 UTC,No Data,SO,SPEC,0.0,WV03,No Data,No Data,No Data
6,17458418010,100436,DAF75,Active,2025-07-29T09:11:10.0000 UTC,2025-07-29T09:11:37.0000 UTC,2025-07-29T09:11:10.0000 UTC,No Data,SO,SPEC,0.0,WV03,No Data,No Data,No Data


## 3. Data Cleaning

In [44]:
df_matching = df_matching.replace("No Data", np.nan)
df_matching["combined"] = (
    df_matching["collectionVehicles"]
    .fillna(df_matching["requestedVehicles"])
    .fillna("No Data")
)
df_matching = df_matching.drop(columns=["requestedVehicles"])

In [45]:
df_mmerged = pd.merge(df_matching, df_result_f, on='orderNumber', how='left')

In [46]:
# order_numbers_with_n = df_result_filtered['Order Number'].astype(str).tolist()
# filtered_order_numbers_with_n = df_matching_optimized['idStore.orderNumber'].astype(str).tolist()

# # Find values that are only in order_numbers_with_n
# only_in_first = set(order_numbers_with_n) - set(filtered_order_numbers_with_n)

# # Find values that are only in filtered_order_numbers_with_n
# only_in_second = set(filtered_order_numbers_with_n) - set(order_numbers_with_n)

# print("Values only in first list:", len(only_in_first))
# print("Values only in second list:", len(only_in_second))

# # Print some examples if there are any differences
# if only_in_first:
#     print("\nExample values only in first list:", list(only_in_first)[:5])
# if only_in_second:
#     print("\nExample values only in second list:", list(only_in_second)[:5])


## 4. Geocoding

In [47]:
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from tqdm import tqdm

In [48]:
def find_country(point, world_gdf):
    # Cek apakah titik berada di dalam poligon negara manapun
    for idx, country in world_gdf.iterrows():
        if point.within(country.geometry):
            return country[country_column]
    
    # Jika titik tidak berada di dalam negara manapun (misalnya di lautan)
    # Cari negara terdekat
    min_distance = float('inf')
    nearest_country = None
    
    for idx, country in world_gdf.iterrows():
        distance = point.distance(country.geometry)
        if distance < min_distance:
            min_distance = distance
            nearest_country = country[country_column]
    
    return f"{nearest_country}"

In [49]:
# Alternatif yang lebih cepat menggunakan spatial join
# Buat GeoDataFrame dari df_merged
world = gpd.read_file('world_admin.geojson')
country_column = 'name'

geometry = [Point(xy) for xy in zip(df_mmerged['longitude'], df_mmerged['latitude'])]

df_geo = gpd.GeoDataFrame(df_mmerged, geometry=geometry, crs="EPSG:4326")

# Lakukan spatial join untuk menemukan negara untuk setiap titik
joined = gpd.sjoin(df_geo, world, how="left", predicate='within')

# Untuk titik yang tidak berada dalam negara manapun (NaN), cari negara terdekat
missing_countries = joined[joined[country_column].isna()]
print(f"Jumlah titik yang tidak berada dalam negara manapun: {len(missing_countries)}")

if len(missing_countries) > 0:
    # Untuk setiap titik yang tidak memiliki negara, cari negara terdekat
    for idx, row in tqdm(missing_countries.iterrows(), total=len(missing_countries)):
        nearest_country = find_country(row.geometry, world)
        joined.at[idx, country_column] = nearest_country

# Salin hasil ke df_merged
df_mmerged['country'] = joined[country_column]

Jumlah titik yang tidak berada dalam negara manapun: 209


100%|██████████| 209/209 [00:06<00:00, 30.90it/s]


In [50]:
# Periksa tipe data kolom latitude dan longitude
print(df_mmerged['latitude'].dtype)
print(df_mmerged['longitude'].dtype)

# Tampilkan beberapa nilai untuk melihat formatnya
print(df_mmerged['latitude'].head(2))
print(df_mmerged['longitude'].head(2))

# Konversi kolom latitude dan longitude ke tipe numerik
# Jika ada masalah dengan format, kita perlu membersihkan data terlebih dahulu
df_mmerged['latitude'] = pd.to_numeric(df_mmerged['latitude'], errors='coerce')
df_mmerged['longitude'] = pd.to_numeric(df_mmerged['longitude'], errors='coerce')

# Periksa apakah ada nilai NaN setelah konversi
print(f"Jumlah nilai NaN di latitude: {df_mmerged['latitude'].isna().sum()}")
print(f"Jumlah nilai NaN di longitude: {df_mmerged['longitude'].isna().sum()}")

# Jika ada nilai NaN, kita bisa menggantinya dengan nilai default atau menghapus baris tersebut
# Opsi 1: Hapus baris dengan NaN
df_mmerged_clean = df_mmerged.dropna(subset=['latitude', 'longitude'])
print(f"Jumlah baris setelah menghapus NaN: {len(df_mmerged_clean)}")

# Sekarang gunakan df_mmerged_clean untuk visualisasi

object
object
0    NaN
1    NaN
Name: latitude, dtype: object
0    NaN
1    NaN
Name: longitude, dtype: object
Jumlah nilai NaN di latitude: 129
Jumlah nilai NaN di longitude: 129
Jumlah baris setelah menghapus NaN: 454


In [51]:
df_mmerged_clean.columns

Index(['orderNumber', 'customerIdentifier', 'customerName', 'status',
       'orderCreateTimestamp', 'orderActiveTimestamp', 'orderSubmitTimestamp',
       'orderCompleteTimestamp', 'orderType', 'productType', 'area',
       'longitude', 'latitude', 'collectionVehicles', 'combined',
       'archiveTasking', 'publicPrivate', 'multistrip', 'responsiveness',
       'completeDate', 'orderDescription', 'catId', 'sqkm', 'charge',
       'chargeS', 'orderNotes', 'country'],
      dtype='object')

In [52]:
df_mmerged_clean['charge'] = (
    df_mmerged_clean['charge']
    .str.replace('$', '', regex=False)   # hapus simbol $
    .str.replace(',', '', regex=False)   # hapus koma
    .astype(float)                       # ubah ke float
)


C:\Users\faisa\AppData\Local\Temp\ipykernel_29548\608083242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mmerged_clean['charge'] = (


In [ ]:
df_mmerged_clean['chargeS'] = pd.to_timedelta(df_mmerged_clean['chargeS']).dt.total_seconds()

C:\Users\faisa\AppData\Local\Temp\ipykernel_29548\593971322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mmerged_clean['chargeS'] = pd.to_timedelta(df_mmerged_clean['chargeS']).dt.total_seconds()


In [54]:
print(df_mmerged_clean['status'].value_counts())


status
Complete     407
Delivered     29
Cancelled     15
Rejected       2
Active         1
Name: count, dtype: int64


In [55]:
df_mmerged_clean

,orderNumber,customerIdentifier,customerName,status,orderCreateTimestamp,orderActiveTimestamp,orderSubmitTimestamp,orderCompleteTimestamp,orderType,productType,...,multistrip,responsiveness,completeDate,orderDescription,catId,sqkm,charge,chargeS,orderNotes,country
8,50354788010,100436,DAF75,Complete,2025-07-28T07:54:49.0000 UTC,2025-07-28T07:55:27.6310 UTC,2025-07-28T07:54:49.0000 UTC,2025-07-28T08:00:06.3090 UTC,AO,OR2A,...,N,Standard,7/28/2025,View-Ready (Standard) OR2A,103001010B13CC00,12.56,12.56,0.027,N,Indonesia
9,50354785010,100436,DAF75,Complete,2025-07-28T07:49:55.0000 UTC,2025-07-28T07:50:20.6890 UTC,2025-07-28T07:49:55.0000 UTC,2025-07-28T07:56:09.1990 UTC,AO,OR2A,...,N,Standard,7/28/2025,View-Ready (Standard) OR2A,B120001100DBB200,12.56,12.56,0.034,N,Indonesia
10,50352143010,100436,DAF75,Complete,2025-07-18T09:24:33.0000 UTC,2025-07-18T09:29:45.5550 UTC,2025-07-18T09:24:33.0000 UTC,2025-07-18T09:32:26.2090 UTC,AO,OR2A,...,N,Fresh,7/18/2025,View-Ready (Standard) OR2A,1030010114745A00,3.86,3.86,0.008,N,Indonesia
11,50352134010,100436,DAF75,Complete,2025-07-18T08:04:41.0000 UTC,2025-07-18T08:05:16.5100 UTC,2025-07-18T08:04:41.0000 UTC,2025-07-18T08:09:38.0330 UTC,AO,OR2A,...,N,Standard,7/18/2025,View-Ready (Standard) OR2A,##############################################...,1.19,1.19,0.002,N,Indonesia
12,50351772010,100436,DAF75,Complete,2025-07-17T06:07:01.0000 UTC,2025-07-17T06:07:31.9980 UTC,2025-07-17T06:07:01.0000 UTC,2025-07-17T06:17:37.3730 UTC,AO,2A,...,N,Standard,7/17/2025,View-Ready (Standard) 2A,104001009C0D5800,79.99,79.99,0.155,N,Indonesia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,17347587010,100436,DAF75,Delivered,2024-10-31T08:59:05.0000 UTC,2024-10-31T08:59:46.0000 UTC,2024-10-31T08:59:05.0000 UTC,2024-11-01T05:02:54.0000 UTC,AO,L3,...,N,Standard,11/1/2024,00:00:00,1040010089D0D200,24.99,24.99,0.048,N,Singapore
579,17347588010,100436,DAF75,Delivered,2024-10-31T08:59:05.0000 UTC,2024-10-31T08:59:47.0000 UTC,2024-10-31T08:59:05.0000 UTC,2024-11-01T05:02:54.0000 UTC,AO,L3,...,N,Standard,11/1/2024,00:00:00,1040010069A58200,25.03,25.03,0.048,N,Indonesia
580,17347585010,100436,DAF75,Delivered,2024-10-31T08:58:56.0000 UTC,2024-10-31T08:59:51.0000 UTC,2024-10-31T08:58:56.0000 UTC,2024-11-01T05:03:13.0000 UTC,AO,L3,...,N,Fresh,11/1/2024,00:00:00,104001009E397400,25.01,25.01,0.048,N,Indonesia
581,17347586010,100436,DAF75,Delivered,2024-10-31T08:58:55.0000 UTC,2024-10-31T08:59:47.0000 UTC,2024-10-31T08:58:55.0000 UTC,2024-11-01T05:03:11.0000 UTC,AO,L3,...,N,Fresh,11/1/2024,00:00:00,10300101067C5900,25.00,25.00,0.054,N,Indonesia


In [56]:
# Import folium untuk visualisasi peta interaktif
import folium
from folium.plugins import MarkerCluster

# Buat peta dasar dengan folium
# Gunakan rata-rata latitude dan longitude sebagai pusat peta
center_lat = df_mmerged_clean['latitude'].mean()
center_lon = df_mmerged_clean['longitude'].mean()
map_viz = folium.Map(location=[center_lat, center_lon], zoom_start=2)

# Tambahkan marker cluster untuk mengelompokkan marker yang berdekatan
marker_cluster = MarkerCluster().add_to(map_viz)

# Tambahkan marker untuk setiap titik dengan popup informasi
for idx, row in df_mmerged_clean.iterrows():
    # Siapkan informasi untuk popup
    popup_text = f"""<b>Order Number:</b> {row['orderNumber']}<br>
                   <b>Customer:</b> {row.get('customerName', 'N/A')}<br>
                   <b>Status:</b> {row.get('status', 'N/A')}<br>
                   <b>Product Type:</b> {row.get('productType', 'N/A')}<br>
                   <b>Area:</b> {row.get('area', 'N/A')} sq km<br>
                   <b>Country:</b> {row.get('country', 'N/A')}<br>"""
    
    # Tambahkan marker dengan popup dan tooltip
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=f"Order: {row['orderNumber']}"
    ).add_to(marker_cluster)

# Simpan peta sebagai file HTML
map_viz.save('order_map.html')

# Tampilkan peta di notebook
from IPython.display import IFrame
IFrame(src='order_map.html', width=800, height=600)

In [57]:
# Import folium untuk visualisasi peta interaktif
import folium
from folium.plugins import MarkerCluster

# Buat peta dasar dengan folium
# Gunakan rata-rata latitude dan longitude sebagai pusat peta
center_lat = df_mmerged_clean['latitude'].mean()
center_lon = df_mmerged_clean['longitude'].mean()
map_viz = folium.Map(location=[center_lat, center_lon], zoom_start=2)

# Tambahkan marker cluster untuk mengelompokkan marker yang berdekatan
marker_cluster = MarkerCluster().add_to(map_viz)

# Tambahkan marker untuk setiap titik dengan popup informasi
for idx, row in df_mmerged_clean.iterrows():
    # Siapkan informasi untuk popup
    popup_text = f"""<b>Order Number:</b> {row['orderNumber']}<br>
                   <b>Customer:</b> {row.get('customerName', 'N/A')}<br>
                   <b>Status:</b> {row.get('status', 'N/A')}<br>
                   <b>Product Type:</b> {row.get('productType', 'N/A')}<br>
                   <b>Area:</b> {row.get('area', 'N/A')} sq km<br>
                   <b>Country:</b> {row.get('country', 'N/A')}<br>"""
    
    # Tambahkan marker dengan popup dan tooltip
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_text, max_width=300),
        tooltip=f"Order: {row['orderNumber']}"
    ).add_to(marker_cluster)

# Tampilkan peta langsung di notebook
from IPython.display import display
display(map_viz)

## 5. Simpan Hasil ke CSV

In [58]:
# # Simpan hasil ke file CSV
# df_matching_optimized.to_csv('filtered_orders_result.csv', index=False)
# print("Hasil telah disimpan ke 'filtered_orders_result.csv'")

In [62]:
df_mmerged_clean.to_excel('ALL_result.xlsx', index=False)


In [60]:
df_a = df_mmerged_clean[df_mmerged_clean['status'] != 'Complete']
df_a.to_csv('filtered_orders_result.csv', index=False)